# Overview of data columns
Programatically sort out data columns 

In [193]:
from glob import glob
from os import path
import os
import pandas as pd
import numpy as np

In [2]:
DATASETS = "../owid-datasets/datasets/*"

def load_data(ds):
    return pd.read_csv(os.path.join("../owid-datasets/datasets/", ds, ds + ".csv"))

In [192]:
ds_dict = {}
ds_shapes = []
all_dtypes = []
for ds in glob(DATASETS):
    ds = os.path.basename(ds)
    df = load_data(ds)
    ds_dict[ds] = df.dtypes
    all_dtypes.extend(df.dtypes)
    ds_shapes.append(df.shape)

In [197]:
all_dtypes = np.array([str(v) for v in set(all_dtypes)])

In [4]:
# How many of these datasets have a year col?
print(sum([1 if "Year" in v else 0 for v in ds_dict.values()]))

992


In [5]:
# How many datasets have a entity col?
print(sum([1 if "Entity" in v else 0 for v in ds_dict.values()]))

992


# Organize the dtypes of each dataset into a dataframe

In [201]:
# create empty df to insert the dtypes for each
from itertools import chain 
y_dim =  set((chain(*[dtypes.index for dtypes in ds_dict.values()])))
x_dim = ds_dict.keys()
df_cols = pd.DataFrame(index=x_dim, columns=y_dim)
df_cols = df_cols.astype('object')
df_cols.head()

,Number of laboratory-confirmed MDR-TB cases identified (conf_mdr),Energy conversion efficiency (Alexander et al. (2016)),Coal by end use (Miscellaneous),"Share of terrorist incidents with suicide attempt (GDT, 2018)",Expansion as share of forest area,Non-Commercial Flight Distance Record (km) (Wikipedia),Percentage who used ads in print publications,MDG5.B: share of pregnant women receiving antenatal care,Main religion,"Unsafe water source (IHME, 2019)",...,Underweight (NCDRisC (2017)),Biofuels (% sub energy),Upper CI (WWF (2016)),Share of calories from carbohydrates (FAO (2017)),Prevalence of undernourishment (World Bank 2017 & UN FAO SOFI (2018)),"Time allocation, weekend, women","Buildings (per capita) (CAIT, 2020)","Industry (CH4 emissions, CAIT)",Maternal Mortality Ratio (Gapminder (2010) and World Bank (2015)),Fertility_Above_2_Below_3
"Genuine saving estimates by various measures - Blum, Ducoing, McLaughlin (2017)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OECD Social Spending- Other social policy areas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Postage rates - United States Postal Service (2018),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Maddison region definitions (Maddison (2010)),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Active tuberculosis - IHME (2017),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
# enter rows to df_cols (index is the dataset, columns are all the fields in every dataset, column values are what dtype the column is) from ds_dict. 
for ds, cols in ds_dict.items():
    # df_cols[ds, cols.index] = [str(c) for c in cols.values]
    cols = cols.astype('object')
    #print(ds in df_cols.index)
    #print(df_cols[cols.index])
    
    df_cols.loc[ds, cols.index] = cols

In [224]:
# check for cases where there are more than 2 unique values
s = df_cols.describe(include='all').loc['unique', :]

In [237]:
print(s[s > 1].shape)
for c in s[s > 1].index:
    print(df_cols.loc[:, c].unique())

(37,)
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[dtype('O') dtype('int64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('float64') dtype('int64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('float64') dtype('int64')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dtype('O')]
[nan dtype('int64') dtype('float64')]
[nan dtype('int64') dt

Index(['Number of laboratory-confirmed MDR-TB cases identified (conf_mdr)',
       'Energy conversion efficiency (Alexander et al. (2016))',
       'Coal by end use (Miscellaneous)',
       'Share of terrorist incidents with suicide attempt (GDT, 2018)',
       'Expansion as share of forest area',
       'Non-Commercial Flight Distance Record (km) (Wikipedia)',
       'Percentage who used ads in print publications',
       'MDG5.B: share of pregnant women receiving antenatal care',
       'Main religion ', 'Unsafe water source (IHME, 2019)',
       ...
       'Underweight (NCDRisC (2017))', 'Biofuels (% sub energy)',
       'Upper CI (WWF (2016))',
       'Share of calories from carbohydrates (FAO (2017))',
       'Prevalence of undernourishment (World Bank 2017 & UN FAO SOFI (2018))',
       'Time allocation, weekend, women',
       'Buildings (per capita) (CAIT, 2020)', 'Industry (CH4 emissions, CAIT)',
       'Maternal Mortality Ratio (Gapminder (2010) and World Bank (2015))',
     

In [35]:
df.head()

""


In [36]:
df2.head()

,Entity,Year,"Net investment (Blum, Ducoing, McLaughlin (2017))","Green investment (Blum, Ducoing, McLaughlin (2017))","Genuine saving (Blum, Ducoing, McLaughlin (2017))","GSTFP (Blum, Ducoing, McLaughlin (2017))","GDP (Blum, Ducoing, McLaughlin (2017))","Green carbon (Blum, Ducoing, McLaughlin (2017))"
0,Argentina,1900,68.78,67.61,69.36,611.86,2915.58,NaN
1,Argentina,1901,124.01,121.35,123.28,684.19,3077.47,NaN
2,Argentina,1902,84.41,82.08,83.99,556.85,2930.59,NaN
3,Argentina,1903,105.50,103.17,105.54,671.40,3259.57,NaN
4,Argentina,1904,156.33,153.95,156.84,781.99,3509.50,NaN


In [41]:
df2_mod = df2.set_index(["Entity", "Year"])
df2_mod.head()

Net investment (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                                      
Argentina 1900                                              68.78   
          1901                                             124.01   
          1902                                              84.41   
          1903                                             105.50   
          1904                                             156.33   

                Green investment (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                                        
Argentina 1900                                              67.61     
          1901                                             121.35     
          1902                                              82.08     
          1903                                             103.17     
          1904                                             153.95     

                Genuine saving (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                                      
Argentina 1900                                              69.36   
          1901                                             123.28   
          1902                                              83.99   
          1903                                             105.54   
          1904                                             156.84   

                GSTFP (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                             
Argentina 1900                                    611.86   
          1901                                    684.19   
          1902                                    556.85   
          1903                                    671.40   
          1904                                    781.99   

                GDP (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                           
Argentina 1900                                 2915.58   
          1901                                 3077.47   
          1902                                 2930.59   
          1903                                 3259.57   
          1904                                 3509.50   

                Green carbon (Blum, Ducoing, McLaughlin (2017))  
Entity    Year                                                   
Argentina 1900                                              NaN  
          1901                                              NaN  
          1902                                              NaN  
          1903                                              NaN  
          1904                                              NaN

In [52]:
do_while = 0
for ds in glob(DATASETS):
    do_while += 1
    if do_while == 1:
        ds = os.path.basename(ds)
        df = load_data(ds)
        df = df.set_index(["Entity", "Year"])
        continue
        
    ds = os.path.basename(ds)
    df2 = load_data(ds)
    df2 = df2.set_index(["Entity", "Year"])
    if do_while == 2:
        break
    df = pd.merge(df, df2, how="inner") 


In [53]:
df.index

MultiIndex([('Argentina', 1900),
            ('Argentina', 1901),
            ('Argentina', 1902),
            ('Argentina', 1903),
            ('Argentina', 1904),
            ('Argentina', 1905),
            ('Argentina', 1906),
            ('Argentina', 1907),
            ('Argentina', 1908),
            ('Argentina', 1909),
            ...
            (    'World', 1991),
            (    'World', 1992),
            (    'World', 1993),
            (    'World', 1994),
            (    'World', 1995),
            (    'World', 1996),
            (    'World', 1997),
            (    'World', 1998),
            (    'World', 1999),
            (    'World', 2000)],
           names=['Entity', 'Year'], length=1212)

In [51]:
df2.index

MultiIndex([(    'Australia', 1980),
            (    'Australia', 1985),
            (    'Australia', 1990),
            (    'Australia', 1995),
            (    'Australia', 2000),
            (    'Australia', 2005),
            (    'Australia', 2010),
            (    'Australia', 2013),
            (    'Australia', 2014),
            (    'Australia', 2015),
            ...
            ('United States', 1980),
            ('United States', 1985),
            ('United States', 1990),
            ('United States', 1995),
            ('United States', 2000),
            ('United States', 2005),
            ('United States', 2010),
            ('United States', 2013),
            ('United States', 2014),
            ('United States', 2015)],
           names=['Entity', 'Year'], length=360)

In [62]:
%timeit df3 = pd.merge(df, df2, how='inner', on=["Entity", "Year"])

2.75 ms ± 7.89 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
df3 = pd.merge(df, df2, how='inner', on=["Entity", "Year"])

In [96]:
print(df.memory_usage().sum()/ 1000000.0)
print(df2.memory_usage().sum()/ 1000000.0)
print(df3.memory_usage().sum()/ 1000000.0)

18.804028
0.120224
0.00258


In [67]:
df3.shape

(40, 7)

In [69]:
df2.shape

(360, 1)

In [73]:
df3.head()

Net investment (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                                      
Australia 1980                                            1444.69   
          1985                                            1023.40   
          1990                                            1317.72   
          1995                                             800.31   
          2000                                            1365.31   

                Green investment (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                                        
Australia 1980                                             824.16     
          1985                                             250.57     
          1990                                             243.69     
          1995                                            -407.96     
          2000                                             -77.01     

                Genuine saving (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                                      
Australia 1980                                            1521.15   
          1985                                            1096.19   
          1990                                            1065.51   
          1995                                             532.70   
          2000                                             995.23   

                GSTFP (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                             
Australia 1980                                   4298.90   
          1985                                   4119.22   
          1990                                   4532.71   
          1995                                       NaN   
          2000                                       NaN   

                GDP (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                           
Australia 1980                                14442.83   
          1985                                15417.54   
          1990                                17434.84   
          1995                                18565.91   
          2000                                21552.43   

                Green carbon (Blum, Ducoing, McLaughlin (2017))  \
Entity    Year                                                    
Australia 1980                                              NaN   
          1985                                              NaN   
          1990                                              NaN   
          1995                                              NaN   
          2000                                              NaN   

               Other social policy areas  
Entity    Year                            
Australia 1980                       0.2  
          1985                       0.2  
          1990                       0.4  
          1995                       0.2  
          2000                       0.2

In [97]:
# don't have to create a blank df with a do_while
do_while = 0
for ds in glob(DATASETS):
    do_while += 1
    if do_while == 1:
        ds = os.path.basename(ds)
        df = load_data(ds)
        df = df.set_index(["Entity", "Year"])
        continue
        
    ds = os.path.basename(ds)
    df2 = load_data(ds)
    df2 = df2.set_index(["Entity", "Year"])

    df = pd.merge(df, df2, how='left', on=["Entity", "Year"])


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [98]:
df.shape

(1212, 2923)

In [99]:
df.index

MultiIndex([('Argentina', 1900),
            ('Argentina', 1901),
            ('Argentina', 1902),
            ('Argentina', 1903),
            ('Argentina', 1904),
            ('Argentina', 1905),
            ('Argentina', 1906),
            ('Argentina', 1907),
            ('Argentina', 1908),
            ('Argentina', 1909),
            ...
            (    'World', 1991),
            (    'World', 1992),
            (    'World', 1993),
            (    'World', 1994),
            (    'World', 1995),
            (    'World', 1996),
            (    'World', 1997),
            (    'World', 1998),
            (    'World', 1999),
            (    'World', 2000)],
           names=['Entity', 'Year'], length=1212)

Net investment (Blum, Ducoing, McLaughlin (2017))      float64
Green investment (Blum, Ducoing, McLaughlin (2017))    float64
Genuine saving (Blum, Ducoing, McLaughlin (2017))      float64
GSTFP (Blum, Ducoing, McLaughlin (2017))               float64
GDP (Blum, Ducoing, McLaughlin (2017))                 float64
                                                        ...   
Per capita international RPKs - adjusted               float64
Per capita international CO2 - adjusted                float64
Per capita total RPKs - adjusted                       float64
Per capita aviation CO2 - adjusted                     float64
Life Expectancy (OECD)                                 float64
Length: 2923, dtype: object